<a href="https://colab.research.google.com/github/dasharakelova/essay_grading/blob/main/nn.strange%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install razdel

In [4]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a2462bd87f8cbd1c903fa2193fe01ab56149df161d59a6b391d82a3155975331
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [44]:
import gensim.downloader
import pymorphy2

from razdel import tokenize
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import nltk
import random
import string

import bs4 as bs
import urllib.request
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

Токенизация

In [45]:
def razdel_tokenize(text):
    tokens = []
    tokens_raw = list(tokenize(text))
    for token in tokens_raw:
        tokens.append(token.text)

    return tokens

In [7]:
razdel_tokenize('Я царь Шумера и Аккада')

['Я', 'царь', 'Шумера', 'и', 'Аккада']

In [9]:
import json
with open('output.json') as cat_file:
    data = json.load(cat_file)

In [39]:
X_train = []
y_train = []
for key, value in data.items():
    X_train.append(razdel_tokenize(value['text']))
    y_train.append(value['score'])

print(X_train[2:5])

[['Как', 'сохранить', 'в', 'своей', 'душе', 'доброту', 'в', 'ситуации', ',', 'когда', 'мы', 'испытываем', 'жестокие', 'удары', 'судьбы', '?', 'Можно', 'ли', 'оставаться', 'человечным', ',', 'если', 'вокруг', 'много', 'зла', 'и', 'несправедливости', '?', 'Правда', 'ли', ',', 'что', 'именно', 'из', 'детства', 'мы', 'черпаем', 'запасы', 'доброты', 'и', 'любви', '?', 'Именно', 'эти', 'вопросы', 'возникают', 'при', 'чтении', 'текста', 'Даниила', 'Гранина', '.', 'Раскрывая', 'проблему', 'сохранения', 'человечности', 'в', 'тяжёлых', 'жизненных', 'условиях', ',', 'автор', 'опирается', 'на', 'собственные', 'рассуждения', '.', 'Он', 'обращается', 'к', 'речи', 'героя', 'Ф', '.', 'М', '.', 'Достоевского', 'Алёши', 'Карамазова', ',', 'акцентируя', 'наше', 'внимание', 'на', 'том', ',', 'что', 'нет', 'ничего', 'выше', 'и', 'полезнее', ',', 'чем', 'хорошее', 'воспоминание', ',', 'вынесенное', 'из', 'детства', ',', 'из', 'родительского', 'дома', '.', 'Алёша', 'Карамазов', ',', 'а', 'вместе', 'с', 'ним'

Лемматизация

In [11]:
morpher = pymorphy2.MorphAnalyzer()

Word2vec

In [14]:
w2v = gensim.downloader.load('word2vec-ruscorpora-300')

In [40]:
vec = np.array([])
x_train = []
for i in range(len(X_train)):
    vec = np.array([0 for o in range(300)])
    for j in range(len(X_train[i])):
        token = X_train[i][j]
        # print(token)
        parsed = morpher.parse(token)[0]
        lemma, pos = parsed.normal_form, parsed.tag.POS
        # print(type(pos))
        if pos in ['NOUN', 'INFN', 'VERB', 'ADJF']:
            if pos == 'INFN':
                pos = 'VERB'
            if pos == 'ADJF':
                pos = 'ADJ'
        try:
          vec = vec + w2v[f'{lemma}_{pos}']
          # vec = vec/len(X_train[i])

        except:
          ...
    x_train.append(vec)
print(len(x_train))
# print(x_train[:3])

2701


In [ ]:
print(x_train[2:5])

[array([], dtype=float64), array([], dtype=float64), array([], dtype=float64)]


In [41]:
print(len(y_train))

2701


In [42]:
x_test = x_train[-600:]
x_train = x_train[:-600]
y_test = y_train[-600:]
y_train = y_train[:-600]

In [43]:
print(len(y_train))
print(len(x_train))

2101
2101


In [46]:
x_train = torch.tensor(x_train)
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)
y_train = torch.tensor(y_train)

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [58]:
class ThreeLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(ThreeLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [49]:
# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

<ipython-input-49-363f6d7e8244>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train = torch.tensor(x_train, dtype=torch.float32)
<ipython-input-49-363f6d7e8244>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)
<ipython-input-49-363f6d7e8244>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test = torch.tensor(x_test, dtype=torch.float32)
<ipython-input-49-363f6d7e8244>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourc

In [59]:
input_size = x_train.shape[1]  # Assuming features are in columns
hidden_size1 = 128
hidden_size2 = 64
output_size = 1 # Assuming multi-output (adjust if single output)
model = ThreeLayerNN(input_size, hidden_size1, hidden_size2, output_size)
loss_fn = nn.MSELoss()  # Use MSE loss for regression, adjust if needed
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate if needed



In [60]:
num_epochs = 600
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train)

    # Calculate loss
    loss = loss_fn(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/600], Loss: 300.3492
Epoch [20/600], Loss: 18.8179
Epoch [30/600], Loss: 24.8294
Epoch [40/600], Loss: 16.2238
Epoch [50/600], Loss: 16.1648
Epoch [60/600], Loss: 15.4919
Epoch [70/600], Loss: 15.1461
Epoch [80/600], Loss: 14.8201
Epoch [90/600], Loss: 14.6335
Epoch [100/600], Loss: 14.5607
Epoch [110/600], Loss: 14.5179
Epoch [120/600], Loss: 14.4691
Epoch [130/600], Loss: 14.4230
Epoch [140/600], Loss: 14.3812
Epoch [150/600], Loss: 14.3403
Epoch [160/600], Loss: 14.3007
Epoch [170/600], Loss: 14.2622
Epoch [180/600], Loss: 14.2245
Epoch [190/600], Loss: 14.1875
Epoch [200/600], Loss: 14.1511
Epoch [210/600], Loss: 14.1151
Epoch [220/600], Loss: 14.0795
Epoch [230/600], Loss: 14.0442
Epoch [240/600], Loss: 14.0091
Epoch [250/600], Loss: 13.9741
Epoch [260/600], Loss: 13.9392
Epoch [270/600], Loss: 13.9045
Epoch [280/600], Loss: 13.8697
Epoch [290/600], Loss: 13.8350
Epoch [300/600], Loss: 13.8002
Epoch [310/600], Loss: 13.7654
Epoch [320/600], Loss: 13.7305
Epoch [330/600],

In [61]:
with torch.no_grad():
    outputs_test = model(x_test)
    test_loss = loss_fn(outputs_test, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 14.0023
